In [ ]:
#02_logreg_adasyn

import mlflow
import optuna
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import ADASYN
from imblearn.pipeline import make_pipeline

# --- Load Data ---
df = pd.read_csv('reddit_preprocessed.csv')
df = df.drop('Unnamed: 0', axis=1, errors='ignore')
df = df.dropna()
df['clean_comment'] = df['clean_comment'].astype(str)

le = LabelEncoder()
y = le.fit_transform(df['category'])
class_names = [str(cls) for cls in le.classes_]

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    df['clean_comment'], y, test_size=0.2, random_state=42, stratify=y
)

# --- Optuna ---
mlflow.set_experiment("EXP 5 - ML Algos with optuna")
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    params = {
        'C': trial.suggest_float('C', 0.1, 10.0, log=True),
        'solver': 'liblinear',
        'max_iter': 1000
    }

    clf = make_pipeline(
        TfidfVectorizer(max_features=3000, ngram_range=(1, 2)),
        ADASYN(random_state=42),
        LogisticRegression(**params, random_state=42)
    )

    return cross_val_score(clf, X_train_raw, y_train, cv=3, scoring='accuracy').mean()

print("🚀 Tuning Logistic Regression...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# --- Final Training ---
best_params = study.best_params
final_pipeline = make_pipeline(
    TfidfVectorizer(max_features=3000, ngram_range=(1, 2)),
    ADASYN(random_state=42),
    LogisticRegression(**best_params, random_state=42, solver='liblinear')
)

final_pipeline.fit(X_train_raw, y_train)
y_pred = final_pipeline.predict(X_test_raw)

# --- Logging ---
with mlflow.start_run(run_name="LogReg_ADASYN"):
    mlflow.log_params(best_params)

    report = classification_report(y_test, y_pred, target_names=class_names, output_dict=True)

    print("\n📊 Logistic Regression Report:")
    print(classification_report(y_test, y_pred, target_names=class_names))

    mlflow.log_metric("accuracy", report['accuracy'])
    mlflow.log_metric("macro_f1", report['macro avg']['f1-score'])

    for label in class_names:
        if label in report:
            mlflow.log_metric(f"{label}_precision", report[label]['precision'])
            mlflow.log_metric(f"{label}_recall", report[label]['recall'])
            mlflow.log_metric(f"{label}_f1", report[label]['f1-score'])

🚀 Tuning Logistic Regression...

📊 Logistic Regression Report:
              precision    recall  f1-score   support

          -1       0.72      0.73      0.72      1650
           0       0.83      0.89      0.86      2529
           1       0.90      0.84      0.87      3154

    accuracy                           0.83      7333
   macro avg       0.81      0.82      0.82      7333
weighted avg       0.83      0.83      0.83      7333

🏃 View run LogReg_ADASYN at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/5/runs/a4cd8dc65d434470af8201f36d8025a3
🧪 View experiment at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/5


In [2]:
!pip install mlflow dagshub optuna imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/

In [1]:
import dagshub
dagshub.init(repo_owner='Ritk-Raikwar', repo_name='reddit-comment-sentiment-analysis', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=b8173615-add4-4d2a-9b56-40ce2a531af7&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=b82d51fd1b32b8cf3d28800e58f30af41ea73f5b82fb88cf4e5bc827d5d3008b




Accessing as Ritk-Raikwar

Initialized MLflow to track repo "Ritk-Raikwar/reddit-comment-sentiment-analysis"

Repository Ritk-Raikwar/reddit-comment-sentiment-analysis initialized!